In [ ]:
import numpy as np
import dill
import pandas as pd

import datetime as dt
from tqdm import tqdm
from collections import defaultdict, Counter

import warnings

warnings.filterwarnings("ignore")

# pd.set_option('display.max_rows', 3000)
# pd.reset_option('display.max_rows')

In [ ]:
df = pd.read_csv('/New_MIMIC4_p_1.csv')
date_columns = ['ADMITTIME', 'DISCHTIME', 'INTIME', 'OUTTIME']
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df.shape

### integrated data dictionary

In [ ]:
D_iv_ITEMS = pd.read_csv('/3.1/icu/d_items.csv.gz')
D_iv_ITEMS.columns = D_iv_ITEMS.columns.str.upper()

D_iii_ITEMS = pd.read_csv('/mimic-iii-clinical-database-1.4/d_items.csv.gz')
D_iii_ITEMS.columns = D_iii_ITEMS.columns.str.upper()

print(D_iii_ITEMS.shape,D_iv_ITEMS.shape)

D_ITEMS = pd.concat([D_iii_ITEMS[['ITEMID', 'LABEL', 'LINKSTO', 'CATEGORY', 'UNITNAME','PARAM_TYPE']],D_iv_ITEMS[['ITEMID', 'LABEL', 'LINKSTO', 'CATEGORY', 'UNITNAME','PARAM_TYPE']]])
D_ITEMS['CATEGORY'] = D_ITEMS['CATEGORY'].str.upper()
print(D_ITEMS.shape)

D_ITEMS = D_ITEMS.drop_duplicates(subset=['ITEMID','LABEL'],keep='first')
D_ITEMS = D_ITEMS.dropna(subset=['ITEMID','LABEL'])
D_ITEMS = D_ITEMS.sort_values(by='LABEL')
D_ITEMS = D_ITEMS.drop_duplicates(subset=['ITEMID'],keep='first')
print(D_ITEMS.shape)

In [ ]:
D_iv_LABITEMS = pd.read_csv('/3.1/hosp/D_LABITEMS.csv.gz')
D_iv_LABITEMS.columns = D_iv_LABITEMS.columns.str.upper()

D_iii_LABITEMS = pd.read_csv('/mimic-iii-clinical-database-1.4/D_LABITEMS.csv.gz')
D_iii_LABITEMS.columns = D_iii_LABITEMS.columns.str.upper()

print(D_iii_LABITEMS.shape,D_iv_LABITEMS.shape)

D_LABITEMS = pd.concat([D_iii_LABITEMS[['ITEMID', 'LABEL', 'CATEGORY']],D_iv_LABITEMS[['ITEMID', 'LABEL', 'CATEGORY']]])
D_LABITEMS['CATEGORY'] = D_LABITEMS['CATEGORY'].str.upper()

print(D_LABITEMS.shape)
D_LABITEMS = D_LABITEMS.drop_duplicates(subset=['ITEMID','LABEL'],keep='first')
D_LABITEMS['LABEL'].replace(' ', np.nan, inplace=True)
D_LABITEMS['LABEL'].replace('(Albumi','(Albumin)', inplace=True)
D_LABITEMS = D_LABITEMS.dropna(subset=['ITEMID','LABEL'])
D_LABITEMS = D_LABITEMS.sort_values(by='LABEL')
D_LABITEMS = D_LABITEMS.drop_duplicates(subset=['ITEMID'],keep='first')
D_LABITEMS['LINKSTO'] = 'chartevents'
print(D_LABITEMS.shape)

In [ ]:
All_ITEMS = pd.concat([D_ITEMS[['ITEMID', 'LABEL', 'LINKSTO','CATEGORY']], D_LABITEMS])
All_ITEMS = All_ITEMS.sort_values(by='LABEL')
All_ITEMS = All_ITEMS[~(All_ITEMS.CATEGORY == 'ADT')]
del(D_ITEMS,D_LABITEMS)
print(All_ITEMS.shape)

In [ ]:
del(D_iii_ITEMS,D_iv_ITEMS,D_iii_LABITEMS,D_iv_LABITEMS)

### 1. Integrate the deformed ones

In [ ]:
Albumin = ['ALBUMIN','(Albumin)','<Albumin>','Albumin','albumin']
ALT = ['ALT','Alanine Aminotransferase (ALT)']
AST = ['AST','Asparate Aminotransferase (AST)']
PLT = ['Platelet','Platelet Count', 'Platelets', 'PLTS']
Sodium = ['Sodium','Sodium (135-148)']
Sodium_WholeBlood = ['Sodium, Whole Blood']
Magnesium = ['Magnesium', 'Magnesium (1.6-2.6)']
Chloride = ['Chloride', 'Chloride (100-112)']
Phosphorous = ['Phosphorous','Phosphorous(2.7-4.5)']
INR = ['INR','INR (2-4 ref. range)']

NSBP = ['NBP [Systolic]','Non Invasive Blood Pressure systolic']
NDBP = ['NBP [Diastolic]','Non Invasive Blood Pressure diastolic']
NMBP = ['NBP Mean','Non Invasive Blood Pressure mean']

MAP = ["ART BP Mean", "ART MAP", "ART MEAN", "Arterial BP Mean", 
 "Arterial BP Mean #2", "Arterial Blood Pressure mean", 
 "FEM ART MAP", "FEM ART MEAN", "art mean", "arterial mean #2"]
ABP = ["ART BP", "ART Blood Pressure", "Arterial BP", "Arterial Blood Pressure", 
 "FEM ART BP", "Fem Art BP", "L FEM ART BP", "L fem art bp", 
 "R fem art BP", "RAD ART BP", "RT FEMORAL ART BP"]
ASBP = ["ART BP Systolic", "Arterial BP #2 [Systolic]", 
        "Arterial BP [Systolic]", "Arterial Blood Pressure systolic"]
ADBP = ["ART BP Diastolic", "Arterial BP #2 [Diastolic]", 
 "Arterial BP [Diastolic]", "Arterial Blood Pressure diastolic"]

SaO2 = ["ART %O2 saturation (PA Line)", "Arterial O2 Saturation"]
SpO2 = ['Oxygen Saturation', 'PAR-Oxygen saturation','SpO2','O2 saturation pulseoxymetry']
PaO2 = ['Arterial PaO2','pO2','Arterial O2 pressure']

Arterial_pH = ["Art.pH", "Arterial pH", "pH (Art)", "pH (Arterial)"]
Urine_PH = ['Urine PH', 'Urine pH', 'urine PH', 'urine pH', 'urine ph']

Femoral_ArteryLine = ["FEM ART LINE", "L FEM ART LINE", "L fem art line", "femoral ART Line", 
'FEM ART',"R FEM ART LINE", "R fem art line", "RT FEM ART LINE",'fem art', 'femoral ART Line', 'r fem art line', 'rt art line']  

Arterial_PaCO2 = ['Arterial PaCO2', 'Arterial CO2 Pressure']
Arterial_TCO2 = ['Arterial CO2 (Calc)', 'TCO2 (calc) Arterial']

ETCO2 = ['ETCO2', 'EtCO2']

Pain = ['Pain','Pain (0-10)','Pain Level']
Pain_Presence = ['Currently experiencing pain','Pain Present']

Dialysis = ['Dialysis','DIALYSIS']
Dialysis_Output = ['DIALYSIS OUT', 'DIALYSIS OUTPUT', 'DIALYSIS TOTAL OUT','Dialysis Out', 'Dialysis Output.', 'dialysis  out','dialysis out', 'dialysis output']
Hemodialysis_Output = ['HEMODIALYSIS OUT', 'HEMODIALYSIS OUTPUT', 'Hemodialysis OUT', 'Hemodialysis Out','Hemodialysis Output', 'Hemodialysis out', 'hemodialysis out', 'hemodialysis output', 'Hemo dialysis out']
Hemodialysis = ['HEMODIALYSIS', 'HEMODIALYSIS.', 'HemoDialysis', 'Hemodialysis.', 'hemodialysis', 'hemodialysis crystal', 'hemodialysis ultrafe']
Peritoneal_Dialysis = ['PERITONEAL DIALYSIS', 'Peritoneal Dialysis', 'peritoneal dialysis', 'Peritoneal dialysis']

LDH = ['LDH','Lactate Dehydrogenase (LD)']

Temperature_C = ['Temperature', 'Temperature C', 'Temperature C (calc)', 'Temperature Celsius']
Temperature_F = ['Temperature F', 'Temperature F (calc)', 'Temperature Fahrenheit']

Creatine_Kinase = ['CK (CPK)', 'CPK','Creatine Kinase (CK)']

Auto_PEEP = ['AUTOPeeP', 'Auto PeeP', 'Auto-PEEP Level', 'Auto-PEEP level','Autopeep', 'auto-peep', 'autopeep', 'Intrinsic peep','ZAuto Peep Level']
PEEP_Set = ['PEEP Set', 'PEEP set']
Sigh_PEEP = ['PEEP Sigh level', 'SIGH PEEP LEVEL', 'Sigh PEEP level','Sigh Peep', 'Sigh peep level']
Total_PEEP = ['Total PEEP Level', 'total PeeP']

BUN = ['BUN','BUN    (6-20)','BUN (6-20)','Bun','bun','Urea Nitrogen']

Differential_Bands = ['BANDS','Bands','Bands#','Differential-Bands']

Calcium = ['CALCIUM', 'Calcium', 'Calcium, Total', 'Total Calcium', 'calcium', 'Ca','Calcium (8.4-10.2)']
Ionized_Calcium = ['% Ionized Calcium', 'IONIZED CALCIUM', 'Ion Calcium', 'Ionized Calcium','Ionized calcium', 'ionized calcium', 'Free Calcium']
Calcium_Urine = ['Calcium, Urine', 'urine calcium']

ABG_Potassium = ['ABG POTASSIUM', 'ABG Potassium']
Potassium = ['POTASSIUM', 'Potassium', 'potassium','Potassium (3.5-5.3)', 'Potassium  (3.5-5.3)']
Potassium_Urine = ['URINE POTASSIUM', 'Potassium, Urine']
Potassium_WholeBlood = ['Potassium, Whole Blood']

Creatinine = ['Cr','Creatinine','Creatinine   (0-0.7)','Creatinine (0-1.3)']
Creatinine_WholeBlood = ['Creatinine (whole blood)','Creatinine, Blood','Creatinine, Whole Blood']
Creatinine_Urine= ['Urine  Creatinine','Urine Creatinine','urine creatinine','Creatinine, Urine']

FiO2 = ['FIO2','FiO2','Inspired O2 Fraction']
FiO2_Set = ['FIO2 SET','FiO2 Set']

Glucose = ['Blood Glucose', 'BloodGlucose', 'Glucose','Glucose (70-105)']
Glucose_WholeBlood = ['Glucose (whole blood)', 'Glucose (whole blood) (soft)', 'Glucose, Whole Blood']
Fingerstick_Glucose = ['FINGERSTICK GLUCOSE.', 'Fingerstick Glucose', 'Glucose finger stick', 'finger stick glucose', 'fingerstick glucose']

HCO3 = ['BICARBONATE-HCO3', 'HCO3', 'Na Bicarbonate','Sodium Bicarbonate', 'bicarbonate', 'sodium bicarbonate', 'soduim bicarbonate', 'Bicarbonate']

Height = ['Height', 'Height (cm)']
Admission_Weight = ['Admission Weight (Kg)', 'Admit Wt']

PT = ['Prothrombin time', 'PT','PT(11-13.5)']
PTT = ['PTT','PTT(22-35)']

Anion_Gap = ['Anion Gap', 'Anion gap']

Respiratory_Rate = ["Resp Rate", "Resp. Rate", "Respiratory Rate"]
Resp_Spontaneous = ["Resp Rate (Spont)", "Spont Resp rate", "Spont. Resp. Rate", "Respiratory Rate (spontaneous)"]
Respiratory_Rate_Total = ["Resp Rate (Total)", "Respiratory Rate (Total)", "Respiratory Rate (Set)","Respiratory Rate Set"]
High_Resp_Rate = ["HIGH Resp Rate", "High Resp. Rate"]

Bilirubin_total = ["BILI", "Bili", "bili", "TBILI", "tbili", "Total Bili", "Bilirubin", "Bili level", "bili level", "Total Bilirubin", "Bilirubin, Total"]
Direct_Bilirubin = ['Direct Bili','Direct Bili (0-0.3)','Direct Bilirubin','Bilirubin, Direct']
Indirect_Bilirubin = ['Indirect Bili(0-1.0)','Bilirubin, Indirect']

WBC = ['White Blood Cells','WBC','WBC   (4-11,000)','WBC (4-11,000)','WBC 4.0-11.0','WBC Count']
RBC = ['RBC','Red Blood Cells','Red Blood C(3.6-6.2)']
GCS_EyeOpening = ['Eye Opening', 'GCS - Eye Opening']
GCS_MotorResponse = ['GCS - Motor Response', 'Motor Response']
GCS_VerbalResponse = ['GCS - Verbal Response', 'Verbal Response']

In [ ]:
mapping_groups = {
    'Albumin': Albumin,
    'ALT': ALT,
    'AST': AST,
    'PLT':PLT,
    'Sodium':Sodium,
    'Sodium_WholeBlood':Sodium_WholeBlood,
    'Magnesium':Magnesium,
    'Chloride':Chloride,
    'Phosphorous':Phosphorous,
    'INR':INR,
    'NSBP':NSBP,
    'NDBP':NDBP,
    'NMBP':NMBP,
    'MAP': MAP,
    'ABP': ABP,
    'ASBP': ASBP,
    'ADBP': ADBP,
    'SaO2': SaO2,
    'SpO2': SpO2,
    'PaO2':PaO2,
    'Arterial_pH': Arterial_pH,
    'Urine_PH': Urine_PH,
    'Femoral_ArteryLine': Femoral_ArteryLine,
    'Arterial_PaCO2': Arterial_PaCO2,
    'Arterial_TCO2': Arterial_TCO2 ,
    'ETCO2': ETCO2,
    'Pain': Pain,
    'Pain_Presence': Pain_Presence,
    'Dialysis': Dialysis,
    'Dialysis_Output': Dialysis_Output,
    'Hemodialysis_Output': Hemodialysis_Output,
    'Hemodialysis': Hemodialysis,
    'Peritoneal_Dialysis': Peritoneal_Dialysis,
    'LDH': LDH,
    'Temperature_C': Temperature_C,
    'Temperature_F': Temperature_F,
    'Creatine_Kinase': Creatine_Kinase,
    'Auto_PEEP': Auto_PEEP,
    'PEEP_Set': PEEP_Set,
    'Sigh_PEEP': Sigh_PEEP,
    'Total_PEEP': Total_PEEP,
    'BUN': BUN,
    'Differential_Bands': Differential_Bands,
    'Calcium': Calcium,
    'Ionized_Calcium': Ionized_Calcium,
    'Calcium_Urine': Calcium_Urine,
    'ABG_Potassium': ABG_Potassium,
    'Potassium': Potassium,
    'Potassium_Urine': Potassium_Urine,
    'Potassium_WholeBlood':Potassium_WholeBlood,
    'Creatinine': Creatinine,
    'Creatinine_WholeBlood': Creatinine_WholeBlood,
    'Creatinine_Urine': Creatinine_Urine,
    'FiO2': FiO2,
    'FiO2_Set': FiO2_Set,
    'Glucose': Glucose,
    'Glucose_WholeBlood': Glucose_WholeBlood,
    'Fingerstick_Glucose': Fingerstick_Glucose,
    'HCO3': HCO3,
    'Height': Height,
    'Admission_Weight':Admission_Weight,
    'PT': PT,
    'PTT':PTT,
    'Anion_Gap':Anion_Gap,
    'Respiratory_Rate': Respiratory_Rate,
    'Resp_Spontaneous':Resp_Spontaneous,
    'Respiratory_Rate_Total': Respiratory_Rate_Total,
    'High_Resp_Rate': High_Resp_Rate,
    'Bilirubin_total': Bilirubin_total,
    'Direct_Bilirubin': Direct_Bilirubin,
    'Indirect_Bilirubin': Indirect_Bilirubin,
    'WBC': WBC,
    'RBC':RBC,
    'GCS_EyeOpening': GCS_EyeOpening,
    'GCS_MotorResponse': GCS_MotorResponse,
    'GCS_VerbalResponse': GCS_VerbalResponse
}

In [ ]:
replace_dict = {item: key for key, values in mapping_groups.items() for item in values}
All_ITEMS['LABEL'] = All_ITEMS['LABEL'].replace(replace_dict)

In [ ]:
All_ITEMS[All_ITEMS.LABEL == 'Height']

### 2. Integrate exactly the same labels

In [ ]:
same = All_ITEMS[All_ITEMS.duplicated(subset=['LABEL', 'LINKSTO'],keep=False)]
same = (
    same.groupby('LABEL')
    .agg(
        final_ICD_CODE=('ITEMID', lambda x: max(x)),  
        all_ICD_CODEs=('ITEMID', lambda x: list(x))          
    )
    .reset_index()
)

same_map = {
    code: final 
    for final, codes in zip(same['final_ICD_CODE'], same['all_ICD_CODEs']) 
    for code in codes if code!=final
}
All_ITEMS['ITEMID'] = All_ITEMS['ITEMID'].replace(same_map)
All_ITEMS = All_ITEMS.drop_duplicates(subset=['ITEMID','LABEL'],keep='first')
print(All_ITEMS.shape)

In [ ]:
len(pd.unique(All_ITEMS.ITEMID))

In [ ]:
All_ITEMS[All_ITEMS.LABEL == 'Height']

# CHARTEVENTS + LABEVENTS

In [ ]:
#CHARTEVENTS
dataset_path = '/3.1/icu/chartevents.csv.gz'

chunksize = 10000000
Chart = pd.DataFrame()

for chunk in tqdm(pd.read_csv(dataset_path, compression='gzip', header=0, index_col=None, chunksize=chunksize)):

    chunk.columns = chunk.columns.str.upper()
    chunk.rename(columns={'STAY_ID': 'ICUSTAY_ID'}, inplace=True)
    chunk = chunk[['ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUENUM']]
    
    chunk = chunk[chunk['ICUSTAY_ID'].isin(df['ICUSTAY_ID'])]

    chunk = chunk.dropna(subset=['VALUE', 'VALUENUM'], how='all')
    
    if chunk.shape[0] > 0:
        Chart = pd.concat([Chart, chunk], ignore_index=True)
del(chunk)
print(Chart.shape, len(Chart.ITEMID.unique()))

In [ ]:
# LABEVENTS
dataset_path = '/hosp/labevents.csv.gz'

chunksize = 10000000
Lab = pd.DataFrame()
for chunk in tqdm(pd.read_csv(dataset_path, compression='gzip', header=0, index_col=None, chunksize=chunksize)):
    chunk.columns = chunk.columns.str.upper()
    chunk = chunk[['HADM_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUENUM']]
    
    chunk = chunk[chunk['HADM_ID'].isin(df['HADM_ID'])]
    
    chunk = chunk.dropna(subset=['VALUE', 'VALUENUM'], how='all')
    
    if chunk.shape[0] > 0:
        Lab = pd.concat([Lab, chunk], ignore_index=True)
del(chunk)        
print(Lab.shape, len(Lab.ITEMID.unique()))

In [ ]:
Chart['ICUSTAY_ID'] = Chart['ICUSTAY_ID'].astype(int)
Lab['HADM_ID'] = Lab['HADM_ID'].astype(int)
print(Chart.shape, Lab.shape)

Chart = Chart.dropna(subset=['VALUENUM'], how='all')
Lab = Lab.dropna(subset=['VALUENUM'], how='all')
print(Chart.shape, Lab.shape)

In [ ]:
Chart_keep = Chart.copy()
LAB_keep = Lab.copy()

In [ ]:
def pre_icu_ts_data(df):

    within_criteria = df['CHARTTIME'].between(df['INTIME'], df['OUTTIME'])
    df = df[within_criteria].copy()

    df['event_time_from_admit'] = df['CHARTTIME'] - df['INTIME']
    df['start_time'] = df['event_time_from_admit'].dt.total_seconds() / 3600
    df['los'] = (df['OUTTIME'] - df['INTIME']).dt.total_seconds() / 3600

    df = df[df['start_time'] >= 0]

    cols_to_keep = ['ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUENUM',
                    'INTIME', 'OUTTIME', 'event_time_from_admit', 'start_time', 'los']
    df = df[cols_to_keep]

    return df

In [ ]:
LAB_keep = pd.merge(LAB_keep,df[['HADM_ID','ICUSTAY_ID','INTIME', 'OUTTIME']],on='HADM_ID',how='left')
LAB_keep['INTIME'] = pd.to_datetime(LAB_keep['INTIME'])
LAB_keep['OUTTIME'] = pd.to_datetime(LAB_keep['OUTTIME'])
LAB_keep['CHARTTIME'] = pd.to_datetime(LAB_keep['CHARTTIME'])
print(LAB_keep.shape)

LAB_keep = pre_icu_ts_data(LAB_keep)
print(LAB_keep.shape)

In [ ]:
Chart_keep = pd.merge(Chart_keep,df[['ICUSTAY_ID','INTIME', 'OUTTIME']],on='ICUSTAY_ID',how='left')
Chart_keep['INTIME'] = pd.to_datetime(Chart_keep['INTIME'])
Chart_keep['OUTTIME'] = pd.to_datetime(Chart_keep['OUTTIME'])
Chart_keep['CHARTTIME'] = pd.to_datetime(Chart_keep['CHARTTIME'])
print(Chart_keep.shape)

Chart_keep = pre_icu_ts_data(Chart_keep)
print(Chart_keep.shape)

In [ ]:
All_TS = pd.concat([Chart_keep,LAB_keep])
print(Chart_keep.shape,LAB_keep.shape,All_TS.shape)
del(Chart, Lab)

In [ ]:
All_TS['ITEMID'] = All_TS['ITEMID'].replace(same_map)
print(All_TS.shape)
All_TS = All_TS.drop_duplicates(keep='first')
print(All_TS.shape)

In [ ]:
All_TS.head()

In [ ]:
All_TS = pd.merge(All_TS,All_ITEMS[['ITEMID','LABEL']],on='ITEMID',how='left')
print(All_TS.shape,len(All_TS.ITEMID.unique()))

In [ ]:
# Admission Weight (lbs.) - 226531
All_TS.loc[All_TS['LABEL'] == 'Admission Weight (lbs.)', 'VALUENUM'] = (All_TS.loc[All_TS['LABEL'] == 'Admission Weight (lbs.)', 'VALUENUM'] * 0.453592).round(2)
All_TS.loc[All_TS['ITEMID'] == 226531, 'ITEMID'] = 226512
All_TS.loc[All_TS['LABEL'] == 'Admission Weight (lbs.)', 'LABEL'] = 'Admission_Weight'

# Temperature_F - 223761
All_TS.loc[All_TS['LABEL'] == 'Temperature_F', 'VALUENUM'] = ((All_TS.loc[All_TS['LABEL'] == 'Temperature_F', 'VALUENUM'] - 32) * 5/9).round(2)
All_TS.loc[All_TS['ITEMID'] == 223761, 'ITEMID'] = 223762
All_TS.loc[All_TS['LABEL'] == 'Temperature_F', 'LABEL'] = 'Temperature_C'

In [ ]:
#unique_C = All_TS[['ICUSTAY_ID', 'ITEMID']].drop_duplicates()
Citemid_frequency = unique_C.groupby('ITEMID')['ICUSTAY_ID'].count().reset_index()
Citemid_frequency.columns = ['ITEMID', 'ICUSTAY_ID_count']
Citemid_frequency = Citemid_frequency.sort_values(by='ICUSTAY_ID_count', ascending=False)
print(Citemid_frequency.shape)
Citemid_frequency = Citemid_frequency[Citemid_frequency.ICUSTAY_ID_count >= len(All_TS.ICUSTAY_ID.unique()) * 0.3]
Citemid_frequency = pd.merge(Citemid_frequency,All_ITEMS[['ITEMID','LABEL']],on='ITEMID',how='left')
print(Citemid_frequency.shape)

In [ ]:
ADTs = [226515, 226516, 226518, 226228, 226724, 220003, 226544, 226381,227378, 226545, 226543]

alarm_features_to_remove = [
    "Alarms On",'HR Alarm [High]','HR Alarm [Low]',
    'NBP Alarm [High]','NBP Alarm [Low]',
    'Resp Alarm [High]','Resp Alarm [Low]',
    'SpO2 Alarm [High]','SpO2 Alarm [Low]',
    'Heart Rate Alarm - Low',
    'Heart rate Alarm - High',
    'O2 Saturation Pulseoxymetry Alarm - Low',
    'O2 Saturation Pulseoxymetry Alarm - High',
    'Resp Alarm - Low',
    'Resp Alarm - High',
    'Non-Invasive Blood Pressure Alarm - Low',
    'Non-Invasive Blood Pressure Alarm - High',
    'Minute Volume Alarm - High',
    'Minute Volume Alarm - Low',
    'Arterial Blood Pressure Alarm - Low',
    'Arterial Blood Pressure Alarm - High',
    "PSV Level", 
    'Paw High',
    'Vti High',
    'Fspn High',
    'H', 'L', 'I'
]


subjective_features_to_remove = [
    "Richmond-RAS Scale", "Goal Richmond-RAS Scale",
    "Acuity Workload Question 1", "Acuity Workload Question 2", 
    'Is the spokesperson the Health Care Proxy',
    'Any fear in relationships',
    'Sexuality / reproductive problems',
    'Emotional / physical / sexual harm by partner or close relation',
    'Unable to assess psychological',
    'Unable to assess nutrition / education',
    'Social work consult',
    "History of slips / falls", "Home TF",
    "Recreational drug use",  'Orientation',
    "Special diet", "PH (dipstick)", "Specific Gravity (urine)",
    "Braden Sensory Perception", "Braden Mobility", "Braden Moisture",
    "Braden Nutrition", "Braden Activity", "Braden Friction/Shear",
    # 'Strength R Arm', 'Strength L Arm',
    # 'Strength R Leg', 'Strength L Leg',    
    "Activity / Mobility (JH-HLM)", "Gait/Transferring", "Ambulatory aid",
    #"Mental status",
    # "Visual / hearing deficit","Unintentional weight loss >10 lbs.","History of falling (within 3 mnths)",
]


procedural_features_to_remove = [
    'Insulin pump', 'Cough/Deep Breath','Apnea Interval','Cuff Pressure',
    'Intravenous  / IV access prior to admission','Cuff Pressure','Ventilator Mode',
    'Parameters Checked',
    'ST Segment Monitoring On',
    'IV/Saline lock',
    '20 Gauge placed in outside facility',
    '20 Gauge placed in the field',
    '20 Gauge Dressing Occlusive',
    '18 Gauge Dressing Occlusive',
    '18 Gauge placed in outside facility',
    '18 Gauge placed in the field',
    'Multi Lumen placed in outside facility',
    'Arterial Line placed in outside facility',
    'Arterial Line Zero/Calibrate',
    'Ventilator Tank #1',
    'Ventilator Tank #2',
    'Called Out',
    "Subglottal Suctioning", "Eye Care",'High risk (>51) interventions',
    'Secondary diagnosis',"Pressure Ulcer Present",
    "Differential-Basos","Differential-Monos", "Differential-Eos"
]


redundant_unit_features_to_remove = [
    'Glucose finger stick (range 70-100)',
    'Tidal Volume (set)','Tidal Volume (spontaneous)','Minute Volume', # 可由 tidal volume × RR 得出
]

Chart_features_to_remove = (
    alarm_features_to_remove +
    subjective_features_to_remove +
    procedural_features_to_remove +
    redundant_unit_features_to_remove
)
len(Chart_features_to_remove)

In [ ]:
Citemid_frequency = Citemid_frequency[~(Citemid_frequency.LABEL.isin(Chart_features_to_remove))]
Citemid_frequency = Citemid_frequency[~(Citemid_frequency.ITEMID.isin(ADTs))]
Citemid_frequency.shape

In [ ]:
print(len(pd.unique(Citemid_frequency.LABEL.values)),list(Citemid_frequency.ITEMID.values))

In [ ]:
Citemid_frequency[Citemid_frequency.LABEL.duplicated(keep=False)]

In [ ]:
iiis = [228232, 223762, 220045, 220615, 220739, 223901, 225624, 223900, 220210, 52610, 52569, 52028, 220228, 52623, 227457, 220546, 52535, 50882, 51493, 53185, 53186, 51691, 51277, 227073, 220179, 220180, 220181, 220635, 50970, 52035, 225677, 227465, 227466, 227467, 51675, 226512, 52730, 225664, 220224, 50804, 52038, 52040, 220235, 224828, 224639, 787, 742, 198, 87, 225312, 225309, 225310, 53154, 220339, 224690, 224697, 225667, 224695, 224689, 225668, 220602, 227443, 220545, 220645, 227442, 220621, 470, 225690, 220644, 220587, 225625, 224696, 53086, 223834, 225634, 227346, 227341, 226253, 227456, 52452, 227881, 581, 227688, 51994, 53188, 52272, 225641, 52256, 4753, 52225, 53164, 777, 226861, 224409, 225187, 225185, 580, 220632, 7570, 50911, 50816, 225313, 450, 50810, 434, 218, 631, 578, 52455]
len(iiis)

In [ ]:
iviiis = list(pd.unique(list(Citemid_frequency.ITEMID.values)+iiis))
iviiis = list(set(iviiis)-set([578, 450, 580, 581, 198, 742, 227688, 777, 4753, 7570, 787, 434, 87, 631, 470, 218,229323, 53166]))
print(len(iviiis),iviiis)

In [ ]:
All_ITEMS[All_ITEMS.ITEMID.isin(iviiis)].LABEL.values

In [ ]:
All_TS_keep = All_TS[All_TS.ITEMID.isin(iviiis)]
print(All_TS_keep.shape, len(All_TS_keep.ITEMID.unique()))

In [ ]:
print(list(All_TS_keep.ITEMID.unique()))

In [ ]:
del(Citemid_frequency)

In [ ]:
included_itme = All_ITEMS[All_ITEMS.ITEMID.isin(iviiis)]
c_dict = dict(zip(included_itme['ITEMID'], included_itme['LABEL']))
print(All_TS_keep.shape,len(All_TS_keep.ITEMID.unique()))
print(len(c_dict),c_dict)

In [ ]:
def is_numeric(value):
    try:
        float(value)  
        return True
    except (ValueError, TypeError):
        return False


def compute_outlier_imputation(arr, upper_percentile, lower_percentile):
    arr = pd.to_numeric(arr)

    upper_bound = np.percentile(arr, upper_percentile)
    lower_bound = np.percentile(arr, lower_percentile)
    
    arr[arr < lower_bound] = lower_bound
    arr[arr > upper_bound] = upper_bound

    return arr

def outlier_imputation(data, id_attribute, value_attribute, num_col, upper_percentile=98, lower_percentile=2):
    grouped = data.groupby(id_attribute)

    for id_number, group in grouped:
        if id_number not in num_col:
            continue
        
        index = group.index
        values = group[value_attribute].values

        imputed_values = compute_outlier_imputation(values, upper_percentile, lower_percentile)
        
        data.loc[index, value_attribute] = imputed_values

    data = data.dropna(subset=[value_attribute])

    return data

In [ ]:
All_TS_keep.head(2)

In [ ]:
All_TS_keep['is_numeric'] = All_TS_keep['VALUENUM'].apply(is_numeric)

result = All_TS_keep.groupby('ITEMID')['is_numeric'].all().reset_index()
result.rename(columns={'is_numeric': 'is_fully_numeric'}, inplace=True)

num_col = result[result.is_fully_numeric==True].ITEMID.values
print(len(All_TS_keep.ITEMID.value_counts()),len(num_col))

All_TS_keep = outlier_imputation(All_TS_keep, 'ITEMID', 'VALUENUM', num_col, upper_percentile=98, lower_percentile=2)

In [ ]:
All_TS_keep.shape

In [ ]:
del(All_TS)

In [ ]:
los_dict = All_TS_keep.groupby('ICUSTAY_ID')['los'].first().to_dict()
max_los = max(los_dict.values())
print(max_los, max_los*4)
 
final_TS=pd.DataFrame()
t=0

for i in tqdm(np.arange(0, max_los, 0.25)):  
    sub_TS = All_TS_keep[(All_TS_keep['start_time'] >= i) & (All_TS_keep['start_time'] < i + 0.25)]
    
    sub_TS_num = sub_TS[sub_TS['ITEMID'].isin(num_col)].groupby(['ICUSTAY_ID', 'ITEMID']).agg({'VALUENUM': np.nanmedian}).reset_index()
    sub_TS = sub_TS_num

    sub_TS['start_time'] = t

    if final_TS.empty:
        final_TS = sub_TS
    else:
        final_TS = pd.concat([final_TS, sub_TS], ignore_index=True)

    t += 1

In [ ]:
final_TS['ITEMID'] = final_TS['ITEMID'].map(c_dict)
print(final_TS.ITEMID.value_counts())

In [ ]:
final_TS.head()

In [ ]:
print(len(final_TS.ITEMID.unique()),list(final_TS.ITEMID.unique()))

In [ ]:
final_TS[final_TS.ITEMID.isin(['GCS_EyeOpening','GCS_VerbalResponse', 'GCS_MotorResponse'])].head()

In [ ]:
def create_Dict(TS):
    dataDic = {}
    feat = list(TS['ITEMID'].unique())  
    feat = feat + ['GCS_Total']
    
    for hid in tqdm(TS.ICUSTAY_ID.unique()):  
        df2 = TS[TS['ICUSTAY_ID'] == hid] 
        if df2.shape[0] == 0:
            print(f'empty：{hid}')
            break

        else:

            val = df2.pivot(index='start_time', columns='ITEMID', values='VALUENUM')
            
            add_indices = pd.Index(range(df2.start_time.max())).difference(val.index)
            add_df = pd.DataFrame(index=add_indices, columns=val.columns)
            val = pd.concat([val, add_df]) 
            val = val.sort_index() 

            feat_df = pd.DataFrame(columns=list(set(feat) - set(val.columns))) 
            val = pd.concat([val, feat_df], axis=1) 

            val['GCS_Total'] = val[['GCS_EyeOpening', 'GCS_VerbalResponse', 'GCS_MotorResponse']].sum(axis=1, skipna=True)

            val['GCS_Total'] = val['GCS_Total'].replace(0, np.nan)
            
            val = val[feat] 

        # 保存数据到 CSV
        val.to_csv(f'/MIMIC4_TS/CHARTS_IV/{hid}.csv', index=False)

In [ ]:
create_Dict(final_TS)

# TS static concat

In [ ]:
import os

In [ ]:
TS_file = os.listdir('/MIMIC4_TS/CHARTS_IV')
print(len(TS_file), TS_file[0])

In [ ]:
df.shape

In [ ]:
ts_median = pd.DataFrame()

if not TS_file:
    print("none")
else:

    for csv_file in tqdm(TS_file):
        try:
            df = pd.read_csv(f'/MIMIC4_TS/CHARTS_IV/{csv_file}')

            if len(df) > 192:
                df = df.iloc[:192]
            
            medians = {}
            for column in df.columns:
                if df[column].notna().any():  
                    medians[column] = df[column].median()
                else:
                    medians[column] = np.nan 
            
            result_df = pd.DataFrame([medians])
            result_df['ICUSTAY_ID'] = int(csv_file.split('.')[0])
            ts_median = pd.concat([ts_median,result_df])
            
        except Exception as e:
            print(f"{csv_file} {e}\n")

In [ ]:
print(ts_median.shape)

In [ ]:
ts_median.describe()

In [ ]:
ts_median['GCS_Total'].value_counts()

In [ ]:
ts_median = ts_median[~(ts_median[ts_median.drop('ICUSTAY_ID', axis=1).columns].isna().all(axis=1))]
ts_median.shape

In [ ]:
ts_median.to_csv('/IV_ts_static.csv',index=False)